# Azure Pipelines

## Pipeline Tasks
A packaged script or procedure that's been abstracted with a set of inputs.

An Azure Pipelines task abstracts away the underlying details. This abstraction makes it easier to run common build functions, like downloading build tools or packages your app depends on, or to build your project, running Visual Studio or Xcode.<br>


  task: DotNetCoreCLI@2<br>
    displayName: 'Build the project'<br>
    inputs:<br>
      command: 'build'<br>
      arguments: '--no-restore --configuration Release'<br>
      projects: '**/*.csproj'<br>

- task: The DotNetCoreCLI@2 task maps to the dotnet command.
- displayName: Defines the task name that's shown in the user interface.
- inputs: Define values that are passed to the command.
- command: Specifies to run the dotnet build subcommand.
- arguments: Specifies additional arguments to pass to the command.
- projects: specifies which projects to build. 
    - Wildcard pattern **/*.csproj is an example of glob patterns. 
    - The ** part specifies to search the current directory and all child directories. 
    - The *.csproj part specifies any .csproj file.
- The "@" in the task name, for example DotNetCoreCLI@2, refers to the task's version. 
  - Allows gradually migrating to the latest version to take advantage of new features.

## Adding Tasks to Pipelines

1. The Visual Designer  
  a. Drag tasks onto a form  
  b. Configure each task  
1. YAML file  
  a. YAML is a compact format that makes it easy to structure the kind of data that's in configuration files. Typically this YAML file is maintained directly with your app's source code.  
  b. *Pipeline as code refers to the concept of expressing your build definitions as code*

## Mapping Script Commands to Pipeline Tasks

**Reference Documentation**  
__[Pipeline Tasks](https://docs.microsoft.com/en-us/azure/devops/pipelines/tasks/?view=azure-devops  )__

|Script command|	Azure Pipelines task|
|:-|:-|
|npm install|	Npm@1|
|node-sass|	CmdLine@2 or script|
|gulp|	gulp@1|
|echo `date`|	CmdLine@2|
|dotnet restore|	DotNetCoreCLI@2|
|dotnet build|	DotNetCoreCLI@2|
|dotnet publish|	DotNetCoreCLI@2|  

There's no built-in task type that runs node-Sass or prints the date to a file so the CmdLine@2 task is used to run any command.  

1. A Build task defines how a test, build, or deployment step is run.  
  a. A build task can compile source code, run unit tests, deploy an application to a staging environment, and more.

## Setting up DevOps Environment

A public GitHub/Azure DevOps repository is accessible to everyone, whereas a private repository is accessible to you and the people you share it with. In both cases, only collaborators can commit changes to a repository. For open source projects typically both the GitHub repository and the Azure DevOps project are set to public. If your project is closed source, you would likely make both your GitHub repository and your Azure DevOps project private.

__[Public Projects](https://docs.microsoft.com/en-us/azure/devops/organizations/public/about-public-projects?view=azure-devops)__

__[Change Project Visibility - Github](https://help.github.com/github/administering-a-repository/setting-repository-visibility)__  

__[Change Project Visibility - Azure DevOps](https://docs.microsoft.com/en-us/azure/devops/organizations/public/make-project-public)__

## Your First Pipeline

1. Move a work item from boards into Doing from To Do  
1. Create the pipeline  
1. Add build tasks and run the pipeline  
  a. The demands section beneath pool specifies that the pipeline needs npm, the Node.js package manager, installed on the build system.  
  b. Under the steps section are the build tasks that map to each of the script commands identified earlier.  
  c. Azure Pipelines provides built-in build tasks that map to many common build activities.  
  d. Not all build activities map to a built-in task. To run general system commands, you use the CmdLine@2 or script task.  
  e. The pipeline uses the script task because it's a common shortcut for CmdLine@2.
1. Note the elements below:  
  a. (Build.DefinitionName) specifies the name of the build pipeline. For example, "SpaceGame-Web-CI."  
  b. (Build.BuildId) is a numeric identifier for the completed build, like 115.  
  c. (Build.BuildNumber) is name of the completed build. You can configure the format, but by default the build number includes the current date followed by the build number for that day. An example build number is "20190329.1."
1. Publish results of pipeline builds to pipeline  
  a. Store build artifacts in Azure Pipelines so they're available to others after the build completes.   
  c. In .NET, you can package your app as a .zip file. You can then use the built-in PublishBuildArtifacts@1 task to publish the .zip file to Azure Pipelines.
1. Refactor the build configuration to use variables to make the configuration easier to read and keep up to date  - note that the items below are repeated
  a.. Build configuration: Release  
  b. Location of the wwwroot directory: Tailspin.SpaceGame.Web/wwwroot  
  c. .NET SDK version: 5.x
  d. Add the following to create variables:  
      variables:  
      buildConfiguration: 'Release'  
      wwwrootDir: 'Tailspin.SpaceGame.Web/wwwroot'  
      dotnetSdkVersion: '5.x' <br>
To reference these variables, you use the $() syntax just as you do for built-in variables. 

### Build multiple configurations by using templates
You started with a script that performs each build action and mapped each action to its corresponding pipeline task. The output of the pipeline is a .zip file that contains the compiled web app.

In this exercise, you use a template to define build tasks that can build any configuration defined in the project file. <br>
**Templates enable you to define your logic one time and then reuse it several times.**
**Templates combine the content of multiple YAML files into a single pipeline.**  

To use the 'Release' pipline for 'Debug' you would have to recreate the pipeline, simply replacing the instances of 'Release' with 'Debug'.

This gives you the result you're looking for, but you'd need to manually locate and change both variations of each build task for updates. After you added the additional build requirements, you'd also need to create two tasks, one for the Debug configuration and one for Release, to satisfy those requirements.

### Templates?
A template enables you to define common build tasks one time and reuse those tasks multiple times.

You call a template from the parent pipeline as a build step. You can pass parameters into a template from the parent pipeline.

1. The tasks in a template look like the ones you defined earlier to build and publish the app.  
1. In a template file, you use the parameters section instead of variables to define inputs.  
1. In a template file, you use ${{ }} syntax instead of $() to read a parameter's value. When you read a parameter's value, you include the parameters section in its name. For example, ${{ parameters.buildConfiguration }}.

### Call template from pipeline yaml
1. This file looks like the original, except that it replaces the build and publish tasks with calls to the template that performs the same tasks.  
1. You see that the template is called one time for each configuration. To pass the configuration name to the template, each template task uses the parameters argument.


## Resources
__[Pipeline Documentation](https://docs.microsoft.com/en-us/azure/devops/pipelines/?view=azure-devops)__  
__[YAML Schema Reference](https://docs.microsoft.com/en-us/azure/devops/pipelines/yaml-schema?tabs=schema%2Cparameter-schema&view=azure-devops)__  
__[Learn YAML in Munites](https://learnxinyminutes.com/docs/yaml/)__  
__[Book: Continuous Delivery](https://www.oreilly.com/library/view/continuous-delivery-reliable/9780321670250/)__